#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [ ]:
#импортируем потенциально необходимые библиотеки

import pandas as pd
import numpy as np 
import statsmodels.api as sm
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.iolib.table import SimpleTable
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error 
from sklearn.pipeline import make_pipeline

from itertools import product
from sklearn.model_selection import cross_val_score, KFold
from scipy import stats
from scipy.stats import norm
from sklearn.pipeline import Pipeline
import time
import timeit
import datetime
import plotly.express as px


RANDOM_STATE = 12345

In [ ]:
df = pd.read_csv('/datasets/taxi.csv',index_col=[0],parse_dates=[0])
#номер столбца, который будет использоваться в виде индекса:index_col=[0]
#parse_dates=[0] - столбец, который переводим в формат даты

In [ ]:
display(df.head())

In [ ]:
# столбец num_orders представлен в формате int64.
# столбец datetime представлен в формате object.
df.dtypes

In [ ]:
# проверим пропуски
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
display(df.info())

In [ ]:
# проверка последних строчек с помощью метода tail()
display(df.tail())

In [ ]:
# посмотрим на пропуски
display(df.isna())
display(df.isna().sum())

In [ ]:
df['num_orders'].plot(kind='box',fontsize = 10)
plt.title('Количество заказов такси')
plt.show()

У нас есть данные с 10-ти минутным шагом наблюдений, наша задача – изучить изменения за час, мы можем ресемплировать данные(resampling, англ. менять местами):

In [ ]:
print(df["num_orders"].median()) 

In [ ]:
#Для информации:
#DataFrame.resample(rule, axis=0, closed=None, label=None, convention='start', kind=None, loffset=None, base=None,
#on=None, level=None, origin='start_day', offset=None, group_keys=_NoDefault.no_default)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
#ресемплирования данных за период- один час, мы задали новый шаг наблюдений и пересчитываем количество событий за новый период.
df = df.resample('1h').sum()
df.head()

In [ ]:
#среднее по каждому часу
df.resample('h').mean()

In [ ]:
# максимум 
df.resample('1h').max() 

In [ ]:
# визуализируем выборку данных временных рядов на основе "Месяца"( август- востребованный месяц)
df_month = df.resample("M").mean()

fig, ax = plt.subplots(figsize=(10, 7))
ax.bar(df_month['2018':].index, 
       df_month.loc['2018':, "num_orders"], 
       width=20, align='center')

Выводы:

- загружены данные,в датасете представлены данные  за шесть месяцев (26 496 записей) за период: 01.03.2018 - 31.08.2018;
- тип данных в столбцах: num_orders (dtype: int64 - целые числа),столбец datetime - в формате object;
- анализ не выявил пропущенных значений и аномалий.

## Анализ

In [ ]:
#посчитаем заказы
df.sort_values(by='num_orders')

- 2018-08-20, в 02:00:00: 462 заказа. 
- 2018-04-06 в 06:00:00: 0 заказов.

In [ ]:
#Добавим в столбец 'rolling_mean' скользящее среднее с размером окна = 5.Построим графики c марта по август 2018 года и искользящего среднего
data = df['2018-03':'2018-08'].resample('1D').sum()
data['rolling_mean'] = data.rolling(5).mean()
data.plot()

In [ ]:
# разложим временной ряд на тренд и сезонную компоненту

df.sort_index(inplace=True)
data = df['2018-03':'2018-08'].resample('1D').sum()

decomposed = seasonal_decompose(data) 

plt.figure(figsize=(12, 10))
plt.subplot(311)

decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')
plt.tight_layout()


In [ ]:
#построим график сезонной составляющей за первые 15 дней марта 2018

decomposed = seasonal_decompose(df)
plt.title('Seasonality')
decomposed.seasonal['2018-03-01':'2018-03-15'].plot(ax=plt.gca()) 


In [ ]:
#найдём скользящее стандартное отклонение по скользящему окну:
df.sort_index(inplace=True)
data = df['2018-03':'2018-08'].resample('1D').sum()
data['mean'] = df['num_orders'].rolling(5).mean()
data['std'] = df['num_orders'].rolling(5).std()
data.plot() 


In [ ]:
#Возьмем двое суток у двух разных месяцев: март (начало периода) и август(конец периода).

plt.figure(figsize=(18, 4))

plt.subplot(211)
decomposed.seasonal['2018-03-01':'2018-03-02'].plot(color='tab:blue', marker='o', ax=plt.gca())
plt.title('март')

plt.subplot(212)
decomposed.seasonal['2018-08-01':'2018-08-02'].plot(color='tab:orange', linestyle='--',ax=plt.gca())
plt.title('август')
plt.show()


In [ ]:
#Возьмем сутки в марте
plt.figure(figsize=(18, 4))
decomposed.seasonal['2018-03-01':'2018-03-02'].plot(ax=plt.gca(),color='tab:green')
plt.title('Seasonality "day"')
plt.tight_layout()

In [ ]:
#для проверки временной последовательности распределения данных можно применить функцию is_monotonic.
#True = Последовательность монотонно увеличивается
df.index.is_monotonic 

In [ ]:
if df.index.is_monotonic == True :
    print('Даты расположены в хронологическом порядке')
else:
    print('Требуется предобработка')

In [ ]:
df.index.is_unique 

In [ ]:
#функция для создания признаков

def make_features(data, max_lag, rolling_mean_size):
    
    result = df.copy()
    
    result['day'] = df.index.day
    result['dayofweek'] = df.index.dayofweek
    result['hour'] = df.index.hour
    
    for lag in range(1, max_lag + 1):
        result['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    result['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()
    
    return result

In [ ]:
df_new = make_features(df, 24, 4)

Выводы:
- данные ресемплированы (по 1 часу);
- данные достаточно однородные, чтобы искать уникальные значения;
- тренд идет на возрастание, он не стационарный;
- есть суточная сезонность;
- больше всего заказов такси приходится на 00 часов(имеет смысл увеличить количество автомобилей в это время), минимальное количество заказов в районе 6-7 часов утра.
- дни недели по количеству заказов отличаются: учитываем день недели.

## Обучение

In [ ]:
# обучим разные модели с различными гиперпараметрами,разделим на тестовую выборку размером 10% от исходных данных

df_new.dropna(inplace=True)

X = df_new.drop('num_orders', axis=1)
y = df_new['num_orders']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.1)

In [ ]:
tscv = TimeSeriesSplit(n_splits=4)

In [ ]:
#Размеры выборок:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# создаем функцию и подбираем лучшую модель с помощью Гридсерча

def fit_model(estimator, param_grid, X_train, y_train):
  
    model = GridSearchCV(estimator=estimator, 
                            param_grid=param_grid, 
                            n_jobs=-1, 
                            cv=tscv,
                            scoring='neg_root_mean_squared_error')

    model.fit(X_train, y_train)

    best_index = model.best_index_
    best_rmse = round(model.cv_results_['mean_test_score'][best_index], 1)

    print(f"Best RMSE: {abs(best_rmse)}")
    print(f"Best params: {model.best_params_}")

    return model.best_estimator_

***LinearRegression***


In [ ]:
%%time

lr_estimator = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LinearRegression())
])

scores = cross_val_score(lr_estimator, X_train, y_train, scoring='neg_root_mean_squared_error', cv=tscv)

rmse = scores.mean()

display(f"RMSE: {abs(rmse):.1f}")


***RandomForestRegressor***

In [ ]:
%%time

rf_param_grid = {
    'n_estimators': list(range(40, 100, 10)),
    "max_depth": list(range(5, 12, 2)),
}

rf_best_model = fit_model(
    estimator=RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid=rf_param_grid,
    X_train=X_train,
    y_train=y_train
)

***LGBMRegressor***

In [ ]:
%%time
lgbm_param_grid = {
    "n_estimators": range(70, 200, 15), 
    "max_depth": range(2, 11, 4),
    'learning_rate': [0.01, 0.05, 0.1]
}

lgbm_best_model = fit_model(
    estimator=LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid=lgbm_param_grid,
    X_train=X_train,
    y_train=y_train
)

## Тестирование

In [ ]:
prognostic = lgbm_best_model.predict(X_test)
rmse = mean_squared_error(y_test, prognostic) ** 0.5
print(f"RMSE на тестовой: {rmse:.1f}")

In [ ]:
prognostic = pd.Series(prognostic, index=y_test.index)

plt.figure(figsize=(10, 5))
y_test.plot(label='true',color='tab:red', title='Test')
prognostic.plot(label='predicted',color='tab:green')
plt.legend()
plt.show()

Выводы: провели анализ и подготовили данные для моделей, обучили модели. Модель LGBMRegressor дает лучшие показатели RMSE(значение метрики RMSE на тестовой выборке должно быть не больше 48).